In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lifelines
from lifelines import KaplanMeierFitter
%matplotlib inline

In [2]:
data = pd.read_csv("MockPatientDatabaseOscar.csv", sep=";", encoding='latin-1')
data.head()

,Numero_paciente,Genero,IMC,Regimenafiliacion,Anemia,Hipercalcemia,ERC_Leve,ERC_moderada,ERC_severa,ERC_dialisis,...,AdherenciaTtoMM1Mto1,Oportunidadtratamiento,Dias_suspendidos,Dias_en_terapia,Fecha_respuesta_clinica,RespuestaClinica,Evento,time,TtoMM1,TtoMM2
0,1001,F,"21,3",Contributivo,NO,NO,NO,NO,NO,SI,...,33.0,64,0,189,7/12/2018,NaN,0,189,L(rd),L
1,1002,F,"24,4",Subsidiado,NO,NO,NO,NO,SI,NO,...,25.0,94,0,246,11/05/2019,NaN,0,246,BLd(rs),BLd
2,1003,M,"21,8",Contributivo,NO,NO,NO,SI,NO,NO,...,5.0,66,29,300,15/02/2018,NaN,0,329,BLd(rm),BLd
3,1004,F,"26,2",Contributivo,NO,NO,NO,SI,NO,NO,...,77.0,65,0,334,3/11/2019,NaN,0,334,Ld(rm),Ld
4,1005,M,"28,1",Contributivo,NO,NO,NO,SI,NO,NO,...,NaN,SD,0,SD,25/06/2019,NaN,0,611,BCd,BCd


# Nettoyer les données

In [3]:
# Explore les données doublantes
data_duplicate = data.Numero_paciente.duplicated()
data[data_duplicate]

,Numero_paciente,Genero,IMC,Regimenafiliacion,Anemia,Hipercalcemia,ERC_Leve,ERC_moderada,ERC_severa,ERC_dialisis,...,AdherenciaTtoMM1Mto1,Oportunidadtratamiento,Dias_suspendidos,Dias_en_terapia,Fecha_respuesta_clinica,RespuestaClinica,Evento,time,TtoMM1,TtoMM2
12,1012,M,"24,55775234",Contributivo,NO,NO,SI,NO,NO,NO,...,74.0,62,0,10,20/10/2019,EP,1,10,BLd,BLd
17,1016,F,"29,64268547",Básico,NO,NO,NO,NO,NO,NO,...,NaN,43925,0,SD,2/03/2020,EP,1,61,BLd,BLd
25,1023,M,"20,33832645",Contributivo,SI,NO,NO,NO,NO,NO,...,NaN,NaN,0,SD,25/01/2020,EP,1,56,DKd,DKd
31,1028,M,"31,13898513",Contributivo,SI,NO,NO,NO,NO,NO,...,50.0,49,0,83,29/04/2019,EP,1,140,DLd,DLd
35,1031,M,"31,57756239",Básico,NO,SI,NO,NO,NO,NO,...,25.0,85,0,69,19/11/2018,EP,1,159,DKd,DKd
41,1036,F,"20,42835711",Contributivo,NO,NO,NO,NO,NO,NO,...,50.0,91,0,52,30/05/2019,EP,1,150,BLd,BLd
43,1037,M,"29,24210864",Contributivo,NO,SI,NO,NO,NO,NO,...,75.0,81,0,104,28/08/2018,EP,1,188,DLd,DLd
45,1038,M,"19,70553242",Básico,NO,NO,NO,NO,NO,NO,...,75.0,68,0,133,17/05/2019,EP,1,205,DLd,DLd
47,1039,F,"16,70620468",Subsidiado,NO,NO,NO,NO,NO,NO,...,90.0,83,0,52,7/04/2019,EP,1,52,DLd,DLd
49,1040,F,"20,19557823",Contributivo,NO,NO,NO,NO,NO,NO,...,100.0,79,0,47,1/04/2020,EP,1,133,DLd,DLd


In [4]:
# Extraire des Numero_paciente doublants en une liste
liste_doublant = data[data_duplicate].Numero_paciente.unique()
liste_doublant

array([1012, 1016, 1023, 1028, 1031, 1036, 1037, 1038, 1039, 1040],
      dtype=int64)

In [5]:
#Montrer les lignes qui contiennt les Numero_paciente doublants et la colonne Evento
doublant = data.loc[data.Numero_paciente.isin(liste_doublant), ['Numero_paciente','Evento']]
doublant

,Numero_paciente,Evento
11,1012,0
12,1012,1
16,1016,0
17,1016,1
24,1023,0
25,1023,1
30,1028,0
31,1028,1
34,1031,0
35,1031,1


## Drop les lignes doublantes

In [6]:
# On garde des lignes qui ont Evento sont égal à 1, et supprime d'autres doublants
data.drop_duplicates(subset=['Numero_paciente'], keep='last', inplace=True, ignore_index=True)
data.loc[data.Numero_paciente.isin(liste_doublant)]

,Numero_paciente,Genero,IMC,Regimenafiliacion,Anemia,Hipercalcemia,ERC_Leve,ERC_moderada,ERC_severa,ERC_dialisis,...,AdherenciaTtoMM1Mto1,Oportunidadtratamiento,Dias_suspendidos,Dias_en_terapia,Fecha_respuesta_clinica,RespuestaClinica,Evento,time,TtoMM1,TtoMM2
11,1012,M,"24,55775234",Contributivo,NO,NO,SI,NO,NO,NO,...,74.0,62,0,10,20/10/2019,EP,1,10,BLd,BLd
15,1016,F,"29,64268547",Básico,NO,NO,NO,NO,NO,NO,...,NaN,43925,0,SD,2/03/2020,EP,1,61,BLd,BLd
22,1023,M,"20,33832645",Contributivo,SI,NO,NO,NO,NO,NO,...,NaN,NaN,0,SD,25/01/2020,EP,1,56,DKd,DKd
27,1028,M,"31,13898513",Contributivo,SI,NO,NO,NO,NO,NO,...,50.0,49,0,83,29/04/2019,EP,1,140,DLd,DLd
30,1031,M,"31,57756239",Básico,NO,SI,NO,NO,NO,NO,...,25.0,85,0,69,19/11/2018,EP,1,159,DKd,DKd
35,1036,F,"20,42835711",Contributivo,NO,NO,NO,NO,NO,NO,...,50.0,91,0,52,30/05/2019,EP,1,150,BLd,BLd
36,1037,M,"29,24210864",Contributivo,NO,SI,NO,NO,NO,NO,...,75.0,81,0,104,28/08/2018,EP,1,188,DLd,DLd
37,1038,M,"19,70553242",Básico,NO,NO,NO,NO,NO,NO,...,75.0,68,0,133,17/05/2019,EP,1,205,DLd,DLd
38,1039,F,"16,70620468",Subsidiado,NO,NO,NO,NO,NO,NO,...,90.0,83,0,52,7/04/2019,EP,1,52,DLd,DLd
39,1040,F,"20,19557823",Contributivo,NO,NO,NO,NO,NO,NO,...,100.0,89,0,13,30/07/2020,EP,1,112,P,P


In [7]:
# Après avoir supprimé les doublant de Numero_paciente, il y a 40.
data

,Numero_paciente,Genero,IMC,Regimenafiliacion,Anemia,Hipercalcemia,ERC_Leve,ERC_moderada,ERC_severa,ERC_dialisis,...,AdherenciaTtoMM1Mto1,Oportunidadtratamiento,Dias_suspendidos,Dias_en_terapia,Fecha_respuesta_clinica,RespuestaClinica,Evento,time,TtoMM1,TtoMM2
0,1001,F,"21,3",Contributivo,NO,NO,NO,NO,NO,SI,...,33.0,64,0,189,7/12/2018,NaN,0,189,L(rd),L
1,1002,F,"24,4",Subsidiado,NO,NO,NO,NO,SI,NO,...,25.0,94,0,246,11/05/2019,NaN,0,246,BLd(rs),BLd
2,1003,M,"21,8",Contributivo,NO,NO,NO,SI,NO,NO,...,5.0,66,29,300,15/02/2018,NaN,0,329,BLd(rm),BLd
3,1004,F,"26,2",Contributivo,NO,NO,NO,SI,NO,NO,...,77.0,65,0,334,3/11/2019,NaN,0,334,Ld(rm),Ld
4,1005,M,"28,1",Contributivo,NO,NO,NO,SI,NO,NO,...,NaN,SD,0,SD,25/06/2019,NaN,0,611,BCd,BCd
5,1006,M,"19,7",Contributivo,NO,SI,NO,SI,NO,NO,...,60.0,57,0,128,26/12/2018,NaN,0,128,BLd(rm),BLd
6,1007,F,"20,3",Subsidiado,NO,NO,SI,NO,NO,NO,...,NaN,SD,0,SD,29/03/2018,NaN,0,514,Observacion,Observacion
7,1008,F,"19,5",Contributivo,NO,NO,SI,NO,NO,NO,...,45.0,65,0,240,12/09/2019,NaN,0,240,BTd,BTd
8,1009,F,"24,3",Contributivo,NO,NO,NO,NO,NO,NO,...,93.0,97,0,281,3/08/2018,NaN,0,281,BCd,BCd
9,1010,M,"28,5",Subsidiado,SI,NO,NO,NO,NO,NO,...,2.0,70,22,159,20/03/2019,EE,0,181,Ld,Ld


## Explore les colonnes 

In [33]:
# La colonne Genero
data.Genero.unique()

array(['F', 'M'], dtype=object)

In [34]:
# la colonne IMC
data.IMC.unique()

array(['21,3', '24,4', '21,8', '26,2', '28,1', '19,7', '20,3', '19,5',
       '24,3', '28,5', '17,2', '24,55775234', '21,0', '22,0', '21,5',
       '29,64268547', '23,4', '27,8', '24,0', '20,33832645', '22,5',
       '19,4', '25,5', '17,7', '31,13898513', '26,5', '21,6',
       '31,57756239', '24,2', '26,7', '28,2', '20,42835711',
       '29,24210864', '19,70553242', '16,70620468', '20,19557823'],
      dtype=object)

In [91]:
# Remplacer , par .
data.IMC = data.IMC.str.replace(",",".", regex=False)
data.IMC
# On va changer un type de donnée object de la colonne en float64 dans l'application web

0            21.3
1            24.4
2            21.8
3            26.2
4            28.1
5            19.7
6            20.3
7            19.5
8            24.3
9            28.5
10           17.2
11    24.55775234
12           21.0
13           22.0
14           21.5
15    29.64268547
16           23.4
17           27.8
18           28.5
19           24.0
20           22.0
21           23.4
22    20.33832645
23           22.5
24           19.4
25           25.5
26           17.7
27    31.13898513
28           26.5
29           21.6
30    31.57756239
31           21.0
32           24.2
33           26.7
34           28.2
35    20.42835711
36    29.24210864
37    19.70553242
38    16.70620468
39    20.19557823
Name: IMC, dtype: object

In [35]:
# La colonne Regimenafiliacion	
data.Regimenafiliacion.unique()

array(['Contributivo', 'Subsidiado', 'Básico'], dtype=object)

In [36]:
# La colonne Anemia
data.Anemia.unique()

array(['NO', 'SI'], dtype=object)

In [37]:
# La colonne Hipercalcemia
data.Hipercalcemia.unique()

array(['NO', 'SI'], dtype=object)

In [38]:
# La colonne ERC_Leve
data.ERC_Leve.unique()

array(['NO', 'SI'], dtype=object)

In [39]:
# La colonne ERC_moderada
data.ERC_moderada.unique()

array(['NO', 'SI'], dtype=object)

In [40]:
# La colonne ERC_severa
data.ERC_severa.unique()

array(['NO', 'SI'], dtype=object)

In [ ]:
# La colonne ERC_dialisis
data.ERC_dialisis.unique()

In [41]:
# La colonne Lesiones_oseas
data.Lesiones_oseas.unique()

array(['NO', 'SI'], dtype=object)

In [42]:
# La colonne Infecciones_recurrentes
data.Infecciones_recurrentes.unique()

array(['NO', 'SI'], dtype=object)

In [43]:
# La colonne Fragilidad
data.Fragilidad.unique()

array(['SI', 'NO'], dtype=object)

In [8]:
# les valeurs remplies dans la la colonne  FISHdel17p1, elle sont censées être NO et SI
data.FISHdel17p1.unique()

array(['NO', 'SD', 'SI'], dtype=object)

In [11]:
# Remplacer la valuer SD par SI
data.FISHdel17p1 = data.FISHdel17p1.str.replace('SD','SI',regex=False)
data.FISHdel17p1.unique()

array(['NO', 'SI'], dtype=object)

In [12]:
# Pareillement , la colonne FISHt_1114 est censée avoir des valurs soit SI soit NO
data.FISHt_1114.unique()

array(['SI', 'NO', 'SD'], dtype=object)

In [13]:
# Remplacer la valuer SD par SI
data.FISHt_1114 = data.FISHt_1114.str.replace('SD','SI', regex=False)
data.FISHt_1114.unique()

array(['SI', 'NO'], dtype=object)

In [14]:
# Pareillement , la colonne FISHt414 est censée avoir des valurs soit SI soit NO
data.FISHt414.unique()

array(['NO', 'SD', 'SI'], dtype=object)

In [15]:
# Remplacer la valuer SD par SI
data.FISHt414 = data.FISHt414.str.replace('SD','SI', regex=False)
data.FISHt414.unique()

array(['NO', 'SI'], dtype=object)

In [16]:
# Pareillement , la colonne FISHamp1q211 est censée avoir des valurs soit SI soit NO
data.FISHamp1q211.unique()

array(['NO', 'SD', 'SI'], dtype=object)

In [17]:
# Remplacer la valuer SD par SI
data.FISHamp1q211 = data.FISHamp1q211.str.replace('SD','SI', regex=False)
data.FISHamp1q211.unique()

array(['NO', 'SI'], dtype=object)

In [18]:
# Pareillement , la colonne FISHother est censée avoir des valurs soit SI soit NO
data.FISHother.unique()

array(['NO', 'SD', 'SI'], dtype=object)

In [19]:
# Remplacer la valuer SD par SI
data.FISHother = data.FISHother.str.replace('SD','SI', regex=False)
data.FISHother.unique()

array(['NO', 'SI'], dtype=object)

In [31]:
# Explore la colonne SubclasificacionplataformaMM
print(data.SubclasificacionplataformaMM.unique())
data.SubclasificacionplataformaMM.value_counts()
# On va remplacer des valuer manquantes (nan) par l'application web

['Riesgo Alto' 'No candidato trasplante' 'No clasificable'
 'Candidato Trasplante' 'Candidato trasplante' nan 'Riesgo No Alto']


No candidato trasplante    18
Riesgo Alto                 5
Candidato Trasplante        4
Riesgo No Alto              3
Candidato trasplante        2
No clasificable             1
Name: SubclasificacionplataformaMM, dtype: int64

In [50]:
data.SubclasificacionplataformaMM.mode()

0    No candidato trasplante
Name: SubclasificacionplataformaMM, dtype: object

In [51]:
# La colonne ISSPlataforma1
data.ISSPlataforma1.unique()

array(['No aplica', 'Estadio 3', 'No clasificable', 'Estadio 2',
       'No estadificable', 'Estadio 1'], dtype=object)

In [52]:
# La colonne CoadOseo1 ,ici on ne traite pas les valuers nan (vides) car il existe la seule valeur non null 
# alors l'information non suffissante
data.CoadOseo1.unique()

array(['Acido selendronico', nan], dtype=object)

In [51]:
# La colonne AdherenciaTtoMM1Mto1
data.AdherenciaTtoMM1Mto1.unique()

array(['No aplica', 'Estadio 3', 'No clasificable', 'Estadio 2',
       'No estadificable', 'Estadio 1'], dtype=object)

In [61]:
# La colonne Oportunidadtratamiento dans laquelle les valuers sont censées être numérique
data.Oportunidadtratamiento.unique()

array(['64', '94', '66', '65', 'SD', '57', '97', '70', '62', '52', '67',
       '43925', '77', '80', '85', nan, '79', '75', '59', '49', '83', '45',
       '96', '98', '91', '81', '68', '89'], dtype=object)

In [65]:
# Remplcaer SD par nan
data.loc[data.Oportunidadtratamiento == 'SD', 'Oportunidadtratamiento'] = np.nan

In [67]:
# Transform object type de la colonne en numérique
data.Oportunidadtratamiento = pd.to_numeric(data.Oportunidadtratamiento)
data.Oportunidadtratamiento.dtypes

dtype('float64')

In [68]:
data.Oportunidadtratamiento.median()

77.0

In [69]:
data.Oportunidadtratamiento.mean()

1363.8529411764705

In [70]:
data.Oportunidadtratamiento.mode()

0    65.0
1    85.0
Name: Oportunidadtratamiento, dtype: float64

#### On prends le median car le mean (moyen) est trop élevé à cause d'une seule valeur aberrante et le median est au milieu de deux modes pour remplcer les valeurs nulles.

In [75]:
data.Oportunidadtratamiento.fillna(77, inplace=True)
data.Oportunidadtratamiento

0        64.0
1        94.0
2        66.0
3        65.0
4        77.0
5        57.0
6        77.0
7        65.0
8        97.0
9        70.0
10       77.0
11       62.0
12       77.0
13       52.0
14       67.0
15    43925.0
16       77.0
17       57.0
18       77.0
19       80.0
20       85.0
21       77.0
22       77.0
23       79.0
24       75.0
25       85.0
26       59.0
27       49.0
28       83.0
29       45.0
30       85.0
31       80.0
32       96.0
33       98.0
34       65.0
35       91.0
36       81.0
37       68.0
38       83.0
39       89.0
Name: Oportunidadtratamiento, dtype: float64

In [76]:
# La colonne Dias_suspendidos
data.Dias_suspendidos.unique()

array([ 0, 29, 22, 30, 27, 23], dtype=int64)

In [77]:
# La colonne Dias_en_terapia
data.Dias_en_terapia.unique()

array(['189', '246', '300', '334', 'SD', '128', '240', '281', '159', '10',
       '179', '216', '186', '157', '180', '165', '219', '155', '215',
       '259', '83', '236', '103', '69', '269', '255', '260', '194', '52',
       '104', '133', '13'], dtype=object)

In [78]:
# Modifier le type de la colonne Dias_en_terapia en mumérique (error = coerce)
data.Dias_en_terapia = pd.to_numeric(data.Dias_en_terapia, errors='coerce')
data.Dias_en_terapia

0     189.0
1     246.0
2     300.0
3     334.0
4       NaN
5     128.0
6       NaN
7     240.0
8     281.0
9     159.0
10      NaN
11     10.0
12      NaN
13    179.0
14    216.0
15      NaN
16    186.0
17    157.0
18      NaN
19    180.0
20    165.0
21    165.0
22      NaN
23    219.0
24    155.0
25    215.0
26    259.0
27     83.0
28    236.0
29    103.0
30     69.0
31    269.0
32    255.0
33    260.0
34    194.0
35     52.0
36    104.0
37    133.0
38     52.0
39     13.0
Name: Dias_en_terapia, dtype: float64

### On va traiter les valuers manquantes de cette colonne dans l'application web

In [79]:
# La colonne Fecha_respuesta_clinica
data.Fecha_respuesta_clinica.unique()

array(['7/12/2018', '11/05/2019', '15/02/2018', '3/11/2019', '25/06/2019',
       '26/12/2018', '29/03/2018', '12/09/2019', '3/08/2018',
       '20/03/2019', '7/10/2018', '20/10/2019', '24/03/2019',
       '23/05/2019', '28/06/2019', '2/03/2020', '18/06/2019', '4/03/2019',
       '29/04/2018', '23/11/2019', '14/12/2018', '26/10/2018',
       '25/01/2020', '7/01/2018', '9/03/2018', '26/03/2018', '25/04/2019',
       '29/04/2019', '9/11/2019', '15/06/2018', '19/11/2018',
       '11/05/2018', '5/11/2019', '19/10/2018', '30/05/2019',
       '28/08/2018', '17/05/2019', '7/04/2019', '30/07/2020'],
      dtype=object)

In [80]:
# La colonne Fecha_respuesta_clinica, les valuers nan sont NA , ici on laisse NA 
data.RespuestaClinica.unique()

array([nan, 'EE', 'EP', 'RCE'], dtype=object)

In [81]:
# La colonne Evento
data.Evento.unique()

array([0, 1], dtype=int64)

In [82]:
# La colonne time
data.time.unique()

array([189, 246, 329, 334, 611, 128, 514, 240, 281, 181, 284,  10, 107,
       179, 216,  61, 186, 187, 314, 180, 165, 192,  56, 219, 155, 215,
       259, 140, 103, 159, 269, 255, 260, 150, 188, 205,  52, 112],
      dtype=int64)

In [83]:
# La colonne TtoMM1
data.TtoMM1.unique()

array(['L(rd)', 'BLd(rs)', 'BLd(rm)', 'Ld(rm)', 'BCd', 'Observacion',
       'BTd', 'Ld', 'BLd', 'DKd', 'L', 'DLd', 'P'], dtype=object)

In [84]:
# La colonne TtoMM2
data.TtoMM2.unique()

array(['L', 'BLd', 'Ld', 'BCd', 'Observacion', 'BTd', 'DKd', 'DLd', 'P'],
      dtype=object)

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Numero_paciente               40 non-null     int64  
 1   Genero                        40 non-null     object 
 2   IMC                           40 non-null     object 
 3   Regimenafiliacion             40 non-null     object 
 4   Anemia                        40 non-null     object 
 5   Hipercalcemia                 40 non-null     object 
 6   ERC_Leve                      40 non-null     object 
 7   ERC_moderada                  40 non-null     object 
 8   ERC_severa                    40 non-null     object 
 9   ERC_dialisis                  40 non-null     object 
 10  Lesiones_oseas                40 non-null     object 
 11  Infecciones_recurrentes       40 non-null     object 
 12  Fragilidad                    40 non-null     object 
 13  FISHdel

In [96]:
data.Genero.dtypes

dtype('O')

In [97]:
data.Numero_paciente.dtypes

dtype('int64')

In [98]:
data.Dias_en_terapia.dtypes

dtype('float64')

In [99]:
#Test remplacer un mot
data.RespuestaClinica.fillna("EE")

0      EE
1      EE
2      EE
3      EE
4      EE
5      EE
6      EE
7      EE
8      EE
9      EE
10     EE
11     EP
12     EE
13     EE
14    RCE
15     EP
16     EE
17     EE
18     EE
19     EE
20     EE
21     EE
22     EP
23     EE
24     EE
25     EE
26     EE
27     EP
28     EE
29     EE
30     EP
31     EE
32     EE
33     EE
34     EE
35     EP
36     EP
37     EP
38     EP
39     EP
Name: RespuestaClinica, dtype: object

In [100]:
data.RespuestaClinica

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9      EE
10     EE
11     EP
12     EE
13     EE
14    RCE
15     EP
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22     EP
23    NaN
24    NaN
25    NaN
26    NaN
27     EP
28    NaN
29    NaN
30     EP
31    NaN
32    NaN
33    NaN
34    NaN
35     EP
36     EP
37     EP
38     EP
39     EP
Name: RespuestaClinica, dtype: object

In [102]:
data.time.mode()

0    216
1    259
Name: time, dtype: int64

In [105]:
val = data.time.mode()
val[1]

259

In [92]:
# Ecrire data traité en fichier CSV
data.to_csv("MockPatientDatabaseOscarModif.csv", sep=";", encoding="utf_8", index=False, header=True, decimal='.')